In [9]:
# testing uncentered-trained RNN 
# importing pytorch
import torch


In [3]:
# running data config file
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Remote boilerplate/model_boilerplate_remote.py'


In [12]:
# running  model reconstructor file
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Tuning/uncentered_RNN_tuning/RNN_reconstructor.py'

EXACT PARAMETERS FROM BENCHMARK.PY

    projection_amplitude_in = 1.2
    projection_amplitude_out = 1.2
    bionet_params = {
        'target_steps': 150,
        'max_steps': 10,
        'exp_factor': 50,
        'tolerance': 1e-20,
        'leak': 1e-2
    }
    weight_label = 'Interaction'
    source_label = 'TF'
    target_label = 'Gene'
    input_layer.weights.requires_grad = False
    prescale_weights(target_radius=0.8) was applied before training
    
LOADING WITH EXACT TRAINING PARAMETERS


/home/christianl/Zhang-Lab/Zhang Lab Code/Tuning/uncentered_RNN_tuning/RNN_reconstructor.py:402: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net=net,


FileNotFoundError: [Errno 2] No such file or directory: '/home/christian/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt'

In [ ]:
# reconstructing trained RNN
model_dict = '/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt'

loaded_RNN = load_model_from_checkpoint(model_dict, node_names=  , net= , X_in= x_test, y_out= y_test, device='cpu' )

Loading checkpoint...


/tmp/ipykernel_2716497/1100306121.py:339: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/s

Model dimensions: 16371 nodes, 1197 ligands, 16100 TFs
⚠️  Using generic node names
✓ Reconstructed network: 1153904 edges
⚠️  Using dummy input data
⚠️  Using dummy output data
Initializing model...
Loading weights...
✅ Model loaded successfully!
